In [6]:
import langgraph
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver

from typing_extensions import TypedDict
from typing import Dict, Any
# from utils.agents.agent_state import AgentState
from utils.common.logger import LOGGER

In [7]:
class AgentState(TypedDict):
    state: str
    results: Dict[str, Any]

In [19]:
def start_agent(state: AgentState):
    LOGGER.info("Starting the agent-assist")
    state['state'] = 'start'
    return state

# It is not an agent, it is a redirector
def check_user_qry(state: AgentState):
    LOGGER.info("Checking user query")
    state['state'] = 'check_user_qry'
    if "python" in state['results']['user']:
        return "python"
    elif "sql" in state['results']['user']:
        return "sql"
    else:
        return "error"

def write_python_code_agent(state: AgentState):
    LOGGER.info("Writing python code")
    state['state'] = 'write_python_code'
    state['results']["write_python_code"] = "print('Hello World')"
    return state

def write_sql_code_agent(state: AgentState):
    LOGGER.info("Writing sql code")
    state['state'] = 'write_sql_code'
    state['results']["write_sql_code"] = "SELECT * FROM table_name"
    return state

def summary_agent(state: AgentState):
    LOGGER.info("Summarizing the code")
    state['state'] = 'sumary'
    state['results']["summary"] = "This is a summary"
    return state

def error_agent(state: AgentState):
    LOGGER.error("Error in the user query")
    state['state'] = 'error'
    return state

def end_agent(state: AgentState):
    LOGGER.info("Ending the agent-assist")
    state['state'] = 'end'
    return state

In [20]:
class MyAgent:
    def __init__(self, thread_id=None):
        self.config = None
        self.app = None
        self.build(thread_id)

    def build(self, thread_id):
        workflow = StateGraph(AgentState)

        # Nodes
        workflow.add_node('start', start_agent)
        workflow.add_node('write_python_code', write_python_code_agent)
        workflow.add_node('write_sql_code', write_sql_code_agent)
        workflow.add_node('summary', summary_agent)
        workflow.add_node('error', error_agent)
        workflow.add_node('end', end_agent)

        # Edges
        workflow.add_conditional_edges(
            'start', 
            check_user_qry,
            {
                "python": "write_python_code",
                "sql": "write_sql_code",
                "error": "error"
            }
        )
        workflow.add_edge('write_python_code', 'summary')
        workflow.add_edge('write_sql_code', 'summary')
        workflow.add_edge('error', 'end')
        workflow.add_edge('summary', 'end')
        

        # Compile
        workflow.set_entry_point('start')
        memory = MemorySaver()
        self.app = workflow.compile(checkpointer=memory)
        self.config = {"configurable":{"thread_id":str(thread_id)}}

    def get_recent_state_snap(self):
        snap = self.app.get_state(self.config).values.copy()
        return snap
    
    def get_graph(self):
        graph = self.app.get_graph(xray=True)
        return graph
    
    def continue_flow(self, state):
        self.app.invoke(state, config=self.config)
        return self.get_recent_state_snap()

In [21]:
MY_AGENT = MyAgent(thread_id=1)


In [22]:
graph = MY_AGENT.get_graph()
print(graph.draw_ascii())

                            +-----------+                        
                            | __start__ |                        
                            +-----------+                        
                                  *                              
                                  *                              
                                  *                              
                              +-------+                          
                            ..| start |..                        
                         ...  +-------+  ...                     
                    .....           .       .....                
                 ...                 .           ...             
              ...                    .              .....        
+-------------------+       +----------------+           ..      
| write_python_code |       | write_sql_code |            .      
+-------------------+       +----------------+            .      
          

In [27]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a python code",
        "write_python_code": "",
        "write_sql_code": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 03:29:56,272 - INFO ==> Starting the agent-assist
2025-03-20 03:29:56,274 - INFO ==> Checking user query
2025-03-20 03:29:56,275 - INFO ==> Writing python code
2025-03-20 03:29:56,275 - INFO ==> Summarizing the code
2025-03-20 03:29:56,276 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': 'Write a python code',
  'write_python_code': "print('Hello World')",
  'write_sql_code': '',
  'summary': 'This is a summary'}}

In [28]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a sql code",
        "write_python_code": "",
        "write_sql_code": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 03:30:01,742 - INFO ==> Starting the agent-assist
2025-03-20 03:30:01,743 - INFO ==> Checking user query
2025-03-20 03:30:01,744 - INFO ==> Writing sql code
2025-03-20 03:30:01,745 - INFO ==> Summarizing the code
2025-03-20 03:30:01,746 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': 'Write a sql code',
  'write_python_code': '',
  'write_sql_code': 'SELECT * FROM table_name',
  'summary': 'This is a summary'}}

In [ ]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a ruby code",
        "write_python_code": "",
        "write_sql_code": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 03:30:12,375 - INFO ==> Starting the agent-assist
2025-03-20 03:30:12,376 - INFO ==> Checking user query
2025-03-20 03:30:12,379 - ERROR ==> Error in the user query
2025-03-20 03:30:12,380 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': 'Write a ruby code',
  'write_python_code': '',
  'write_sql_code': '',
  'summary': ''}}